# Prophet

In [ ]:
# import packages

import pandas as pd
import numpy as np
from fbprophet import Prophet

In [ ]:
Prophet?

## Steel

In [ ]:
# import steel production

df_prod = pd.read_csv('../data/raw/Industry/SteelHistorical_datetime.csv')
df_prod.head()

In [ ]:
df_prod.tail()

In [ ]:
df_macro = pd.read_csv('../data/raw/Industry/MacroHistorical_datetime.csv')

In [ ]:
df_macro.head()

In [ ]:
df_macro.tail()

In [ ]:
# create per capita

df_macro['GDP_per_capita'] = df_macro['GDP'].div(df_macro['Population'])

In [ ]:
# merge dataframes

df = pd.merge(df_prod,df_macro,how='left',on=['Economy','Year'])

In [ ]:
# Select AUS
df = df.set_index('Economy')
df = df.loc['AUS']

In [ ]:
# make Year ds and use only AUS
df =df.reset_index()
df.head()

In [ ]:
df.tail()

In [ ]:
# create natural log of production per capita as a feature

df['ln_prod_per_cap'] = df['SteelConsumption'].div(df['Population'])
df['ln_prod_per_cap'] = np.log(df['ln_prod_per_cap'])

df['ln_GDP_per_cap'] = np.log(df['GDP_per_capita'])

df.head()

In [ ]:
df = df.rename(columns={"Economy": "e", "Year": "ds","ln_prod_per_cap":"y"})
df.head()

In [ ]:
m = Prophet(daily_seasonality=False,
            weekly_seasonality=False,
            yearly_seasonality=False,
            seasonality_mode='multiplicative',
            growth='linear')

m.add_regressor('ln_GDP_per_cap')
m.fit(df)

In [ ]:
# add future GDP

df_future_macro = pd.read_csv('../data/raw/Industry/MacroAssumptions_datetime.csv')
df_future_macro.head()

In [ ]:
# create per capita

df_future_macro['GDP_per_capita'] = df_future_macro['GDP'].div(df_future_macro['Population'])

In [ ]:
# select only AUS

df_future_macro = df_future_macro.set_index('Economy')
df_future_macro = df_future_macro.loc['AUS']
df_future_macro = df_future_macro.reset_index()
df_future_macro.tail()

In [ ]:
# create natural log of GDP per capita
df_future_macro['ln_GDP_per_cap'] = np.log(df_future_macro['GDP_per_capita'])
df_future_macro.head()

In [ ]:
# extract GDP_per_capita for 1980-2050

df1 = pd.DataFrame(columns = ['ln_GDP_per_cap'])
df2 = pd.DataFrame(columns = ['ln_GDP_per_cap'])

df1['ln_GDP_per_cap'] = df['ln_GDP_per_cap']
df2['ln_GDP_per_cap'] = df_future_macro['ln_GDP_per_cap']
#df1 = df['GDP_per_capita']
#df2 = df_future_macro['GDP_per_capita']

In [ ]:
df2.info()

In [ ]:
df_future = pd.concat([df1,df2],
                     ignore_index=True)

In [ ]:
df_future.info()

In [ ]:
m.make_future_dataframe?

In [ ]:
future = m.make_future_dataframe(periods=34,
                                 freq = 'Y',
                                 include_history=True)
future.head()

In [ ]:
future.tail()

In [ ]:
future['ln_GDP_per_cap']=df_future['ln_GDP_per_cap']

In [ ]:
future.head()

In [ ]:
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

# Use Own Function with SciKit-Learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
def run_regression(models, economies, df, x, y):
    """
    Perform linear regression for one or multiple economies.
    economy = list of economies
    models = {economy: LinearRegression() for economy in economies}
    The function returns a dictionary of economy-model pairs. That is,
    each economy will have its own set of coefficients.
    """
    for economy, model in models.items():
            (model.fit(df.loc[economy, x],
                df.loc[economy, y]))
    return models  

In [ ]:
def run_prediction(models, economies, df):
    """
    Use coefficients from run_regression to generate predictions.
    Pass a dataframe df with the X and Y data. 
    ResultsColumn = name of prediction results
    """
    df_list =[]
    # run predictions
    for economy, model in models.items():
            # make prediction
            prediction = model.predict(df.loc[economy,:])
            
            # adding to the input df
            df2 = df.loc[economy,:]
            df2.insert(loc=0,column="Results",value=prediction)
            df2.insert(loc=0,column='Economy',value=economy)
            df2 = df2.reset_index()
            df2 = df2.set_index(['Economy','Year'])
            df_list.append(df2)

    # combine individual economy dataframes to one dataframe
    dfResults = pd.concat(df_list, sort=True)
    return dfResults

### import steel production

In [ ]:
df_prod = pd.read_csv('../data/raw/Industry/SteelHistorical.csv',
                     index_col=['Economy','Year'])
df_prod.head()

In [ ]:
df_macro = pd.read_csv('../data/raw/Industry/MacroHistorical.csv',
                      index_col=['Economy','Year'])
df_macro.head()

### create features

In [ ]:
df_macro['GDP_per_capita'] = df_macro['GDP'].div(df_macro['Population'])

In [ ]:
df = pd.merge(df_prod,df_macro,how='left',on=['Economy','Year'])

In [ ]:
df['ln_prod_per_cap'] = df['SteelConsumption'].div(df['Population'])
df['ln_prod_per_cap'] = np.log(df['ln_prod_per_cap'])

df['ln_GDP_per_cap'] = np.log(df['GDP_per_capita'])

df.head()

In [ ]:
economies = df.index.unique(level=0)
economies

In [ ]:
models = {economy: LinearRegression(fit_intercept=True) for economy in economies}

In [ ]:
x = ['ln_GDP_per_cap']
y = ['ln_prod_per_cap']

In [ ]:
run_regression(models,economies,df,x,y)

### make predictions

In [ ]:
df_future_macro = pd.read_csv('../data/raw/Industry/MacroAssumptions.csv',
                             index_col=['Economy','Year'])
df_future_macro['GDP_per_capita'] = df_future_macro['GDP'].div(df_future_macro['Population'])
df_future_macro.head()

In [ ]:
df_future_macro['ln_GDP_per_cap'] = np.log(df_future_macro['GDP_per_capita'])
df2 = df_future_macro.drop(['GDP', 'Population','GDP_per_capita'], axis=1)
df2.head()

In [ ]:
results = run_prediction(models, economies, df2)
results = results.rename(columns={"Results":"ln_steel_consumption"})
results.head()

### plot results

In [ ]:
x = results.loc['AUS']
x.head()

In [ ]:
fig, ax = plt.subplots()
ax.plot(x.index,x[['ln_steel_consumption']], 'o', label="Predictions")

# Prophet Function

In [ ]:
import pandas as pd
import numpy as np
from fbprophet import Prophet

In [ ]:
df_prod = pd.read_csv('../data/raw/Industry/SteelHistorical.csv')
df_prod.head()

In [ ]:
df_prod['ds']=pd.to_datetime(df_prod['Year'],format='%Y')
df_prod.head()

In [ ]:
df_prod = df_prod.set_index(['Economy'])
df_prod.head()

In [ ]:
df_macro = pd.read_csv('../data/raw/Industry/MacroHistorical.csv')
df_macro.head()

In [ ]:
df_macro['ds']=pd.to_datetime(df_macro['Year'],format='%Y')
df_macro = df_macro.set_index(['Economy'])
df_macro.head()

In [ ]:
df_macro['GDP_per_capita'] = df_macro['GDP'].div(df_macro['Population'])
df = pd.merge(df_prod,df_macro,how='left',on=['Economy','ds','Year'])

In [ ]:
df.head()

In [ ]:
df['ln_prod_per_cap'] = df['SteelConsumption'].div(df['Population'])
df['ln_prod_per_cap'] = np.log(df['ln_prod_per_cap'])

df['ln_GDP_per_cap'] = np.log(df['GDP_per_capita'])

df = df.rename(columns={"ln_prod_per_cap":"y"})

df.head()

In [ ]:


models = {economy: m for economy in economies}

In [ ]:
economies = df.index.unique()
economies

In [ ]:
def run_regression(economies, df):
    """
    Perform linear regression for one or multiple economies.
    economy = list of economies
    models = {economy: LinearRegression() for economy in economies}
    The function returns a dictionary of economy-model pairs. That is,
    each economy will have its own set of coefficients.
    """
    models={}
    m = Prophet(daily_seasonality=False,
    weekly_seasonality=False,
    yearly_seasonality=False,
    seasonality_mode='multiplicative',
    growth='linear')
    for economy in economies:
        m.add_regressor('ln_GDP_per_cap')
            
    return m.fit(df.loc[economy])  

In [ ]:
run_regression(economies, df)

In [ ]:
models